# OpenAI API 6 — RAG + LangChain 


## 학습 목표
- RAG(Retrieval-Augmented Generation)의 개념과 구조 이해
- OpenAI Embeddings API로 벡터화 및 코사인 유사도 계산 실습
- LangChain 설치 및 핵심 컴포넌트(LLM, PromptTemplate, Chain) 이해
- FAISS/Chroma 기반 벡터 검색 + RetrievalQA, ConversationalRetrievalChain 구축
- PDF 문서 기반 RAG 질의응답 시스템 구현



## 1. RAG 개념 및 구조
| 항목 | 설명 |
|---|---|
| 정의 | 외부 지식원(DB/문서/웹)을 검색(Retrieval)해 LLM 생성(Generation)에 반영하는 아키텍처 |
| 핵심 구성 | Retriever(검색기) + Generator(생성기) |
| 장점 | 최신 정보 반영, 환각 감소, 도메인 특화 정확도 향상 |
| 활용 | 사내 위키 Q&A, 매뉴얼/정책 문서 질의응답, 제품 FAQ 챗봇 |

기본 흐름:
```
사용자 질문 → [Retriever] 관련 문서 검색 → [Generator] 문서 기반 답변 생성
```



## 2. OpenAI Embeddings 실습: 벡터화와 코사인 유사도
아래 예제는 텍스트를 임베딩으로 변환하고, 코사인 유사도로 유사도를 계산합니다.


In [ ]:

from openai import OpenAI
import numpy as np

client = OpenAI()

texts = [
    "OpenAI는 인공지능 연구소입니다.",
    "RAG는 검색 기반 생성 모델 구조를 의미합니다.",
    "벡터 데이터베이스는 문장 임베딩을 저장합니다."
]

# 문장 임베딩 생성
embs = []
for t in texts:
    context_vector = client.embeddings.create(input=t, model="text-embedding-3-small").data[0].embedding
    embs.append(context_vector)
    print(len(context_vector))
    print(context_vector)
# embs = [client.embeddings.create(input=t, model="text-embedding-3-small").data[0].embedding for t in texts]

In [ ]:
def cosine_similarity(a, b):
    a = np.array(a); b = np.array(b)
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# 질의 임베딩
query = "RAG 구조에 대해 설명해줘."
q_emb = client.embeddings.create(input=query, model="text-embedding-3-small").data[0].embedding

# 유사도 계산 및 상위 정렬
scores = [cosine_similarity(q_emb, e) for e in embs]
for idx, score in sorted(enumerate(scores), key=lambda x: x[1], reverse=True):
    print(f"{score:.3f} : {texts[idx]}")



## 3. LangChain 설치
아래 명령으로 필요한 라이브러리를 설치하세요.


In [ ]:

# 터미널이나 노트북에서 실행
# %pip install langchain openai faiss-cpu chromadb pypdf tiktoken
print("필요 패키지: langchain, openai, faiss-cpu, chromadb, pypdf, tiktoken")



## 4. LangChain 기본 구조: LLM + PromptTemplate + Chain
LLM, PromptTemplate, Chain을 사용해 간단한 파이프라인을 구성합니다.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini")

template = "너는 친절한 AI 강사야. 다음 질문에 자세히 답해줘: {question}"
prompt = PromptTemplate(template=template, input_variables=["question"])

chain = LLMChain(llm=llm, prompt=prompt)
reply = chain.run("LangChain의 장점은 무엇인가요?")
print(reply)

In [ ]:
from IPython.display import Markdown
Markdown(reply)


## 5. 두 개의 Chain 연결: SimpleSequentialChain
한 단계의 결과를 다음 단계 입력으로 연결합니다.


In [ ]:

from langchain.chains import SimpleSequentialChain

# 1단계: 요약
prompt_summary = PromptTemplate(
    template="다음 텍스트를 한 문장으로 요약해줘:\n{content}",
    input_variables=["content"]
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary) # llm = ChatOpenAI(model="gpt-4o-mini")

# 2단계: 영어 번역
prompt_translate = PromptTemplate(
    template="다음 문장을 영어로 번역해줘:\n{content}",
    input_variables=["content"]
)
chain_translate = LLMChain(llm=llm, prompt=prompt_translate)

# 두 체인을 순차적으로 연결
seq_chain = SimpleSequentialChain(chains=[chain_summary, chain_translate])

# 예문
text = """
LangChain은 LLM을 활용한 애플리케이션 개발을 위한 강력한 오픈소스 프레임워크입니다.
이 라이브러리는 개발자가 프롬프트 관리, 체인 구성, 에이전트 생성 등 복잡한 작업을
모듈화된 방식으로 쉽게 처리할 수 있도록 돕습니다. 특히, 외부 데이터 소스를 LLM과
연동하는 RAG(검색 증강 생성) 시스템을 구축할 때 그 진가를 발휘하며,
이는 AI 챗봇의 답변 정확도를 크게 향상시킬 수 있습니다.
"""

# 체인 실행 및 결과 출력
print(seq_chain.run(text))

In [ ]:
# 요약만 실시
chain_summary.run(text)

In [ ]:
# 번역만 실시
chain_translate.run(text)


## 6. Embedding + FAISS 벡터DB로 유사 검색
텍스트를 벡터화하여 FAISS에 저장하고, 질의로 유사한 문서를 검색합니다.


#### ※ 점수(Score) 해석 방법

`similarity_search_with_score`가 반환하는 점수는 일반적인 '유사도'와는 조금 다릅니다. 이 점수는 두 벡터 사이의 **거리(distance)** 를 의미합니다.

FAISS의 기본 거리 측정 방식은 **L2 거리(유클리드 거리)** 입니다.

  * **점수가 0에 가까울수록 (즉, 작을수록) 더 유사하다**는 의미입니다.
  * 반대로 점수가 클수록 두 벡터의 거리가 멀어 유사성이 낮다는 뜻입니다.

따라서 위 코드를 실행하면, 검색어와 가장 관련성이 높은 문서가 가장 작은 점수와 함께 출력될 것입니다.

In [ ]:
# !pip install langchain_openai

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# OpenAIEmbeddings 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# --- 확장된 문서 리스트 ---
docs = [
    "RAG는 검색과 생성을 결합하여 LLM의 답변 정확도를 높이는 모델 구조입니다.",
    "FAISS는 Meta에서 개발한, 대규모 벡터를 빠르게 검색하기 위한 라이브러리입니다.",
    "Chroma는 파이썬에서 쉽게 쓸 수 있는 경량 오픈소스 벡터 데이터베이스입니다.",
    "임베딩은 텍스트나 이미지 같은 데이터를 컴퓨터가 이해할 수 있는 숫자 벡터로 변환하는 과정입니다.",
    "벡터 저장소는 임베딩된 벡터들을 저장하고, 빠르고 효율적인 유사도 검색을 가능하게 합니다.",
    "LangChain은 LLM을 활용한 애플리케이션 개발을 돕는 강력한 프레임워크입니다.",
    "에이전트는 LLM이 스스로 판단하여 외부 도구를 사용하는 등 복잡한 작업을 수행하는 능력을 가집니다.",
    "파인튜닝은 사전 학습된 거대 언어 모델을 특정 작업이나 도메인에 맞게 추가로 학습시키는 과정입니다.",
    "시맨틱 검색은 단순한 키워드 일치가 아닌, 문장의 문맥과 의미를 기반으로 검색 결과를 제공합니다.",
    "허깅페이스는 트랜스포머 기반의 다양한 모델과 데이터셋을 공유하는 거대한 플랫폼입니다.",
    "Pinecone은 클라우드 기반의 완전 관리형 벡터 데이터베이스 서비스로, 확장성이 뛰어납니다.",
    "LlamaIndex는 LLM에 외부 데이터를 연결하고 질의하는 것에 특화된 데이터 프레임워크입니다."
]

# Faiss 벡터DB 생성
print("벡터 데이터베이스를 생성 중입니다...")
db = FAISS.from_texts(docs, embeddings)
print("생성 완료!")


# --- 다양한 검색어로 테스트 ---
queries = [
    "벡터 데이터베이스 종류 알려줘",
    "LLM한테 외부 지식을 알려주려면 어떻게 해?",
    "모델을 특정 목적에 맞게 훈련시키는 건 뭐야?"
]

for query in queries:
    print("\n" + "="*40)
    print(f"검색어: '{query}'")
    print("="*40)
    
    # 유사도 검색 실행 (점수와 함께)
    results_with_scores = db.similarity_search_with_score(query, k=3)
    
    # 결과 출력
    for doc, score in results_with_scores:
        print(f"Score: {score:.4f}") # 소수점 4자리까지 출력
        print(f"Content: {doc.page_content}")
        print("-" * 20)


## 7. RetrievalQA로 RAG 파이프라인 구성
벡터DB에서 검색한 문맥으로 LLM이 답하도록 구성합니다.


* RetrievalQA는 LLM이 엉뚱한 대답(환각)을 하는 대신, 주어진 자료에 근거하여 정확하고 신뢰도 높은 답변을 하도록 유도합니다.

### 동작 과정
RetrievalQA 체인은 내부적으로 다음과 같은 단계를 거쳐 작동합니다.

**1. 질문 입력**: 사용자가 질문을 합니다. (예: "FAISS의 장점은 무엇인가요?")

**2. 문서 검색 (Retrieve)**:

* 질문 문장을 벡터로 변환(임베딩)합니다.

* 미리 생성해 둔 벡터 저장소(Vector Store, 예: FAISS)에서 질문 벡터와 가장 유사한(관련성 높은) 문서 조각들을 검색합니다.

**3. 프롬프트 구성 (Augment):**

* 검색된 문서 조각들과 원래 질문을 조합하여 LLM에게 전달할 새로운 프롬프트(Prompt)를 만듭니다.

* 예시:
```yaml
    [Context]
    - FAISS는 Meta에서 개발한... 라이브러리입니다.
    - FAISS는 메모리 효율성이 뛰어납니다...

    [Question]
    FAISS의 장점은 무엇인가요?
```
**4. 답변 생성 (Generate):**

구성된 프롬프트를 LLM(예: GPT-4)에 전달합니다.

LLM은 주어진 Context 내용을 바탕으로 질문에 대한 최종 답변을 생성합니다.

In [ ]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

question = "RAG와 FAISS의 역할을 설명해줘."
res = qa.invoke(question)
res

In [ ]:
print("답변:", res["result"])

In [ ]:
print("\n참고 소스 문서:")


In [ ]:
for d in res["source_documents"]:
    print("-", d.page_content)



## 8. ConversationalRetrievalChain로 대화형 RAG
대화 기록을 메모리로 관리하면서 검색+생성을 수행합니다.


In [ ]:

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conv = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory
)

qs = ["RAG는 무엇인가요?", "FAISS는 왜 쓰나요?", "둘의 관계를 간단히 요약해줘."]
for q in qs:
    out = conv.invoke({"question": q})
    print("Q:", q)
    print("A:", out["answer"], "\n")



## 9. PDF 문서 기반 RAG 질의응답
PyPDFLoader로 PDF를 로드하고, 벡터DB를 구성해 질의응답을 수행합니다.


In [ ]:

from langchain.document_loaders import PyPDFLoader

# 예시 파일명: example.pdf (같은 폴더에 준비)
loader = PyPDFLoader("카카오뱅크323410.pdf")
pages = loader.load()

texts = [p.page_content for p in pages]
pdf_db = FAISS.from_texts(texts, embeddings)
pdf_retriever = pdf_db.as_retriever()

qa_pdf = RetrievalQA.from_chain_type(llm=llm, retriever=pdf_retriever)
print(qa_pdf.invoke("이 문서의 핵심 내용을 요약해줘.")["result"])



## 10. 실습 과제
1) 자신의 문서를 TXT/PDF로 준비해 벡터DB 구축 후 질의응답을 구성하세요.  
2) Retrieval 파라미터(k, score_threshold 등)를 조정해 정확도 변화를 비교하세요.  
3) 프롬프트에 인용 표기 형식(예: [출처: 문서제목])을 강제하여 근거를 포함한 답변을 생성해보세요.  

## 참고 자료
- OpenAI Embeddings API: https://platform.openai.com/docs/api-reference/embeddings
- LangChain 공식 문서: https://python.langchain.com/docs/
- FAISS: https://faiss.ai/
- LangChain PDF QA 튜토리얼: https://python.langchain.com/docs/tutorials/pdf_qa
